### Merge daily dataframes to one single dataframe

In [1]:
# Imports
import pandas as pd
import pickle

In [2]:
# Define the date range
start_date = '2022-10-31'
end_date = '2023-03-12'

# Create a list of dates in YYYY-MM-DD format
dates = pd.date_range(start_date, end_date).strftime("%Y-%m-%d").tolist()
print(f"{len(dates)} dates included ({start_date} to {end_date}).")

133 dates included (2022-10-31 to 2023-03-12).


#### Read the pickle files as dicts and add them to a list

In [3]:
# Loop through the dates and read the pickle files. Each pickle file contains a list of dictionaries, each dictionary contains the details of one route.
processed_list = []
missing = 0
for date in dates:
    # Read the pickle for each date. It contains a list of dictionaries, each dictionary contains the details of one harbor combination.
    try:
        with open(f'../pickles/msc_daily_v2/connections_{date}.pickle', 'rb') as f:
            results_list = pickle.load(f)
    except:
        missing += 1
        print(f"Data for {date} missing!")

    # Unpack the list of dictionaries. Each dictionary contains the details of one route. The "Key" key contains another dictionary with the route details. Flatten it into the main dictionary. Put the new items at the beginning of the dictionary.
    for result in results_list:
        # Flatten the information in the "Keys" dictionary.
        result = (result['Key'] | result)
        result.pop('Key')

        # Create a dictionary with the details that's identical for all routes in the list.
        non_route_specific_results = result.copy()
        non_route_specific_results.pop("Routes")

        # Loop through the list of routes, create a new dictionary for each route and append it to the list.
        routes_details_list = result['Routes']
        for route in routes_details_list:
            # Merge the non-route-specific details with the route-specific details.
            new_dict = non_route_specific_results.copy()
            new_dict["NumberOfLegs"] = len(route["RouteScheduleLegDetails"])
            new_dict = (new_dict | route)
            new_dict["ScrapingDate"] = date
            processed_list.append(new_dict)

Data for 2022-11-10 missing!
Data for 2022-11-23 missing!
Data for 2022-11-24 missing!
Data for 2022-11-25 missing!
Data for 2022-11-26 missing!
Data for 2022-11-27 missing!
Data for 2022-11-28 missing!
Data for 2022-12-14 missing!
Data for 2023-02-07 missing!
Data for 2023-03-09 missing!


#### Create a single dataframe from the list of dictionaries

In [4]:
# Create a dataframe from the list of dictionaries
df = pd.DataFrame(processed_list)

# Drop the duplicate PortOfLoadUnCode and PortOfDischargeUnCode columns.
df = df.drop(columns=["PortOfLoadUnCode", "PortOfDischargeUnCode", "PortOfLoad", "EstimatedDepartureTime", "EstimatedDepartureTimeFormatted"])

# Move the second last and third last columns to the beginning of the dataframe
cols = df.columns.tolist()
cols = [cols[-1]] + cols[-3:-1] + cols[:-3]
df = df[cols]

print(f"Total number of routes: {len(df)}, spread over {len(dates)-missing} days.")

# Assign the optimal data types to the columns.
df = df.convert_dtypes()

# Drop rows which are identical, except for the ScrapingDate column.
rows_before = len(df)
columns_to_check = [col for col in df.columns.to_list() if col not in ["ScrapingDate", "CutOffs", "RouteScheduleLegDetails"]]

df = df.drop_duplicates(subset=columns_to_check, keep='first')

# Print the number of identical rows dropped.
print(f"{rows_before - len(df)} identical rows dropped, {1-(len(df)/rows_before):.2%} of the total number of rows, with {len(df)} remaining.")

Total number of routes: 126918, spread over 123 days.
91353 identical rows dropped, 71.98% of the total number of rows, with 35565 remaining.


#### Rename columns
Translate column names following https://docs.google.com/spreadsheets/d/1MkXMOXTt2jIEqUUjSN4Zx1sN-ISponH9DnHL_O1X_l4/edit#gid=412081908

In [5]:
# Read translation table from Google Sheets
SHEET_ID = '1MkXMOXTt2jIEqUUjSN4Zx1sN-ISponH9DnHL_O1X_l4'
SHEET_NAME = 'Combined'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
name_df = pd.read_csv(url, header=0, on_bad_lines='skip')
name_df.head()

,Combined,Format_unit,Maersk,MSC,Routescanner
0,ScrapingDate,str (YYYY-MM-DD),NaN,ScrapingDate,ScrapingDate
1,ScrapingSite,Name (str),Maersk,MSC,Routescanner
2,Origin,UNLOCCODE (str),Origin,Origin,origin_locode
3,Destination,UNLOCCODE (str),Destination,Destination,destination_locode
4,OriginName,str,NaN,PortOfLoading,origin_name


In [6]:
# Create a dictionary with the MSC values as keys and the Combined values as values.
name_dict = dict(zip(name_df['MSC'], name_df['Combined']))

# Rename the columns of df using the dictionary.
df = df.rename(columns=name_dict)
df.head()

,ScrapingDate,Origin,Destination,MaritimeServiceName,OriginName,DestinationName,RouteHeaderId,PortOfLoadId,PortOfDischargeId,TransitTime,...,CutOffs,EstimatedDepartureDateFormatted,EstimatedDepartureTime,EstimatedArrivalDateFormatted,EstimatedArrivalTime,VesselName,DepartureVoyageNo,EstimatedTotalTransitTimeDays,EstimatedTotalTransitTimeHours,Legs
0,2022-10-31,CLARI,BEANR,LLAMA SERVICE,ARICA,ANTWERP,1857438,456,481,34 days,...,{'ContainerYardCutOffDate': 'Mon 31st Oct 2022...,Thu 3rd Nov 2022,2022-11-03T23:30:00,Tue 6th Dec 2022,2022-12-06T16:15:00,MSC BALTIC III,PM243R,33 days,792,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
1,2022-10-31,CLARI,BEANR,LLAMA SERVICE,ARICA,ANTWERP,1857438,456,481,34 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Fri 11th Nov 2022,2022-11-11T08:00:00,Wed 7th Dec 2022,2022-12-07T11:00:00,MSC BALTIC III,PM244R,26 days,624,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
2,2022-10-31,CLARI,NLRTM,LLAMA SERVICE,ARICA,ROTTERDAM,1857439,456,208,33 days,...,{'ContainerYardCutOffDate': 'Mon 31st Oct 2022...,Thu 3rd Nov 2022,2022-11-03T23:30:00,Mon 5th Dec 2022,2022-12-05T05:15:00,MSC BALTIC III,PM243R,32 days,768,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
3,2022-10-31,CLARI,NLRTM,LLAMA SERVICE,ARICA,ROTTERDAM,1857439,456,208,33 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Fri 11th Nov 2022,2022-11-11T08:00:00,Fri 9th Dec 2022,2022-12-09T01:15:00,MSC BALTIC III,PM244R,28 days,672,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
4,2022-10-31,COBUN,BEANR,AZTEC,BUENAVENTURA,ANTWERP,1856444,451,481,26 days,...,{'ContainerYardCutOffDate': 'Wed 2nd Nov 2022 ...,Sun 6th Nov 2022,2022-11-06T22:00:00,Tue 6th Dec 2022,2022-12-06T16:15:00,VANTAGE,2237W,30 days,720,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."


#### Save the dataframe

In [7]:
# Save the dataframe to a pickle file.
filename = "msc_v2_connections_combined"
df.to_pickle(f"../pickles/{filename}.pickle")
df.to_csv(f"../data/{filename}.csv")

In [8]:
pd.read_pickle("../pickles/msc_v2_connections_combined.pickle")

,ScrapingDate,Origin,Destination,MaritimeServiceName,OriginName,DestinationName,RouteHeaderId,PortOfLoadId,PortOfDischargeId,TransitTime,...,CutOffs,EstimatedDepartureDateFormatted,EstimatedDepartureTime,EstimatedArrivalDateFormatted,EstimatedArrivalTime,VesselName,DepartureVoyageNo,EstimatedTotalTransitTimeDays,EstimatedTotalTransitTimeHours,Legs
0,2022-10-31,CLARI,BEANR,LLAMA SERVICE,ARICA,ANTWERP,1857438,456,481,34 days,...,{'ContainerYardCutOffDate': 'Mon 31st Oct 2022...,Thu 3rd Nov 2022,2022-11-03T23:30:00,Tue 6th Dec 2022,2022-12-06T16:15:00,MSC BALTIC III,PM243R,33 days,792,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
1,2022-10-31,CLARI,BEANR,LLAMA SERVICE,ARICA,ANTWERP,1857438,456,481,34 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Fri 11th Nov 2022,2022-11-11T08:00:00,Wed 7th Dec 2022,2022-12-07T11:00:00,MSC BALTIC III,PM244R,26 days,624,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
2,2022-10-31,CLARI,NLRTM,LLAMA SERVICE,ARICA,ROTTERDAM,1857439,456,208,33 days,...,{'ContainerYardCutOffDate': 'Mon 31st Oct 2022...,Thu 3rd Nov 2022,2022-11-03T23:30:00,Mon 5th Dec 2022,2022-12-05T05:15:00,MSC BALTIC III,PM243R,32 days,768,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
3,2022-10-31,CLARI,NLRTM,LLAMA SERVICE,ARICA,ROTTERDAM,1857439,456,208,33 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Fri 11th Nov 2022,2022-11-11T08:00:00,Fri 9th Dec 2022,2022-12-09T01:15:00,MSC BALTIC III,PM244R,28 days,672,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
4,2022-10-31,COBUN,BEANR,AZTEC,BUENAVENTURA,ANTWERP,1856444,451,481,26 days,...,{'ContainerYardCutOffDate': 'Wed 2nd Nov 2022 ...,Sun 6th Nov 2022,2022-11-06T22:00:00,Tue 6th Dec 2022,2022-12-06T16:15:00,VANTAGE,2237W,30 days,720,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126881,2023-03-12,VNVUT,NLRTM,NEW ORIGAMI,VUNG TAU,ROTTERDAM,2028443,1292,208,27 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Sun 7th May 2023,2023-05-07T18:45:00,Fri 2nd Jun 2023,2023-06-02T09:00:00,MSC MATILDE V,HI317A,26 days,624,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
126889,2023-03-12,VNVUT,BEZEE,NEW ORIGAMI,VUNG TAU,ZEEBRUGGE,2302478,1292,589,38 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Sun 7th May 2023,2023-05-07T18:45:00,Tue 13th Jun 2023,2023-06-13T04:00:00,MSC MATILDE V,HI317A,37 days,888,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
126898,2023-03-12,ARZAE,BEANR,COMMERCIAL FEEDER,ZARATE,ANTWERP,2262776,938,481,42 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Sun 7th May 2023,2023-05-07T20:00:00,Sun 18th Jun 2023,2023-06-18T04:00:00,TBN,TBN,42 days,1008,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
126907,2023-03-12,ARZAE,NLRTM,COMMERCIAL FEEDER,ZARATE,ROTTERDAM,2262674,938,208,34 days,...,"{'ContainerYardCutOffDate': '', 'ReeferCutOffD...",Sun 7th May 2023,2023-05-07T20:00:00,Fri 9th Jun 2023,2023-06-09T17:00:00,TBN,TBN,34 days,816,"[{'LegSequence': 1, 'SingleLegOnly': False, 'L..."
